# **Imports &#8595;**

In [1]:
# !pip uninstall -q -y accelerate transformers torch 
# !pip uninstall -q -y sentence-transformers
# !pip install -q accelerate==0.21.0
# !pip install -q transformers==4.24.0
# !pip install -q pytorch
# !pip install -q sentence-transformers
# !pip show accelerate

In [2]:
import numpy as np
import pandas as pd 
import os
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV

from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
from datasets import Dataset
from transformers import Trainer, TrainingArguments
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from transformers import AutoModelForCausalLM, AutoTokenizer

try:
    import vllm
except:
    !pip install torchvision==0.19.1
    !pip install vllm==0.6.3.post1
    !pip install optimum==1.23.3 autoawq==0.2.7.post2 auto-gptq==0.7.1 bitsandbytes==0.44.1 peft==0.12.0

import kagglehub

# Download latest version
lora_path = kagglehub.model_download("anhvth226/2211-lora-14b/transformers/default")
qw14b_path = kagglehub.model_download("anhvth226/qw14b-awq/transformers/default")

c:\Users\edoua\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


WARNING 01-11 15:53:36 _custom_ops.py:19] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")
INFO 01-11 15:53:36 importing.py:10] Triton not installed; certain GPU-related functions will not be available.


2025-01-11 15:53:38,352	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


# **Load Dataset &#8595;**

In [3]:
DATA_PATH = "datasets/eedi-mining-misconceptions-in-mathematics"
EXTERNAL_DATA_PATH = "datasets/eedi-external-dataset"

# train_df = pd.read_csv(f'{DATA_PATH}/train.csv', index_col='QuestionId')
df_train = pd.read_csv(f"{DATA_PATH}/train.csv").fillna(-1).sample(10, random_state=42).reset_index(drop=True)
# train_df = pd.read_csv(f'{EXTERNAL_DATA_PATH}/all_train.csv', index_col='QuestionId') #this contains the original dataset + an external dataset generated by a LLM
test_df = pd.read_csv(f'{DATA_PATH}/test.csv')
misconceptions_df = pd.read_csv(f'{DATA_PATH}/misconception_mapping.csv')
df_misconception_mapping = pd.read_csv(f"{DATA_PATH}/misconception_mapping.csv")

pd.options.display.max_colwidth = 300
# display(train_df.head(5))
pd.options.display.max_colwidth = 50

# **Data Preprocessing &#8595;**

In [4]:
def clean(example, columns):
    """
    Cleans the example from the Dataset
    Args:
        example: an example from the Dataset
        columns: columns that will be cleaned

    Returns: update example containing 'clean' columns

    """
    for col in columns:
        text = example[f'{col}']

        # Empty text
        if type(text) not in (str, np.str_) or text=='':
            example[f'clean_{col}'] = ''
            return example

        # 'text' from the example can be of type numpy.str_, let's convert it to a python str
        text = str(text).lower()

        # Clean the text
        text = re.sub("\"", " ", text) # removes the " from certain texts
        text = re.sub("\n", " ", text) # removes the multiple "\n" 
        text = re.sub(r"(\\\w+)(\W)", r" \1 \2", text) # matches with the LaTeX commands like "\hline{}",... and transforms them to " \hline {}"
        text = re.sub(r"([\(|\{|\[|\|])", r" \1", text) # matches every opening parenthesis types and puts spaces on their left
        text = re.sub(r"([\)|\}|\]])", r"\1 ", text) # matches every closing parenthesis types and puts spaces on their right
        text = re.sub(r"\\(?![a-zA-Z])", " ", text) # removes every backslash that is not the start of a LaTeX command
        text = re.sub(r"\( | \)", "", text) # removes the parentheses that appear sometimes from nowhere 
        text = re.sub(r"\[ | \]", "", text) # removes the parentheses that appear sometimes from nowhere
        
        text = re.sub(r" +", " ", text) # cleans the double spaces made by above substitutions
        # Update the example with the cleaned text
        example[f'clean_{col}'] = text.strip()
    return example

# testing_data = {
#     'QuestionText': ["This is a question with a newline\nin the middle"],
#     'AnswerAText': ["Answer A\nwith newline and \\table[test]"],
#     'AnswerBText': ["Answer B\nwith newline and \hline(uwo)"],
#     'AnswerCText': ["Answer C\nwith newline and \color{gold}"],
#     'AnswerDText': ["Answer D\nwith newline and \\begin{tabular}"]
# }
# df = pd.DataFrame(testing_data)
# df = df.apply(clean, axis = 1, columns = columns_to_clean)
# display(df.head(1))

columns_to_clean = ['QuestionText', 'AnswerAText', 'AnswerBText', 'AnswerCText', 'AnswerDText']
train_df = train_df.apply(clean, axis = 1, columns = columns_to_clean)

# Adjust column order
new_order = ['ConstructId', 'ConstructName', 'SubjectId', 'SubjectName', 'CorrectAnswer']
for col in columns_to_clean:
    new_order.append(col)
    new_order.append(f'clean_{col}')
new_order.extend(['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId'])
train_df = train_df[new_order]


display_train_df = train_df[['QuestionText', 'clean_QuestionText','AnswerAText', 'clean_AnswerAText', 'AnswerBText', 'clean_AnswerBText', 'AnswerCText', 'clean_AnswerCText', 'AnswerDText', 'clean_AnswerDText']]
pd.options.display.max_colwidth = 300
display(display_train_df.head(1))
pd.options.display.max_colwidth = 50

display(misconceptions_df.head(1))

NameError: name 'train_df' is not defined

# **Reshape Dataset For Training &#8595;**

In [11]:
# train_df columns: QuestionID, ConstructID, ConstructName, CorrectAnswer, SubjectId, SubjectName, QuestionText, Answer[A/B/C/D]Text, Misconception[A/B/C/D]Id

reshaped_data = []
for _, row in train_df.iterrows():
    for answer, misconception_id in zip(
        ['clean_AnswerAText', 'clean_AnswerBText', 'clean_AnswerCText', 'clean_AnswerDText'],
        ['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId']
    ): # turn the data into a format where each datapoint (row) represents an answer choice (i.e there are now 4 datapoints for each question)
        misc_id = int(row[misconception_id]) if not pd.isna(row[misconception_id]) else row[misconception_id]
        reshaped_data.append({
            'QuestionText': row['clean_QuestionText'],
            'AnswerText': row[answer],
            'MisconceptionId': misc_id,
            'MisconceptionText': misconceptions_df.loc[misconceptions_df['MisconceptionId'] == misc_id, 'MisconceptionName'].values[0] if not pd.isna(misc_id) else misc_id,
            'SubjectName': row['SubjectName'],
            'ConstructName': row['ConstructName']
        })

reshaped_df = pd.DataFrame(reshaped_data)
display(reshaped_df.head())

# removed columns: QuestionId, ConstructId, CorrectAnswer, SubjectId
# other changes: Answer[A/B/C/D]Text are now in separate datapoints along with their associated Misconception[A/B/C/D]Texts 

,QuestionText,AnswerText,MisconceptionId,MisconceptionText,SubjectName,ConstructName
0,[\r 3 \times 2+4-5\r \r where do the brackets ...,3 \times (2+4) -5,NaN,NaN,BIDMAS,Use the order of operations to carry out calcu...
1,[\r 3 \times 2+4-5\r \r where do the brackets ...,3 \times 2+ (4-5),NaN,NaN,BIDMAS,Use the order of operations to carry out calcu...
2,[\r 3 \times 2+4-5\r \r where do the brackets ...,3 \times (2+4-5),NaN,NaN,BIDMAS,Use the order of operations to carry out calcu...
3,[\r 3 \times 2+4-5\r \r where do the brackets ...,does not need brackets,1672.0,"Confuses the order of operations, believes add...",BIDMAS,Use the order of operations to carry out calcu...
4,"simplify the following, if possible: \frac {m^...",m+1,2142.0,Does not know that to factorise a quadratic ex...,Simplifying Algebraic Fractions,Simplify an algebraic fraction by factorising ...


# **Sentence Embeddings & OneHot Encoding&#8595;**

In [12]:
# remove NaN values (dropping all datapoints that do not have misconceptions assigned to them)
# P.S. that means we are also deleting all the rows (answer choices) that are correct
# P.P.S. unless somehow there are correct answers that have misconceptions associated with them
print(reshaped_df['MisconceptionId'].isnull().sum())  # 10582 NaN values yikes :/
reshaped_df = reshaped_df.dropna(subset=['MisconceptionId'])
print(reshaped_df['MisconceptionId'].isnull().sum())  # 0 now yippie

10582
0


In [7]:
print(reshaped_df.shape)

(4370, 6)


In [6]:
df_ret = df_train.copy()

In [7]:
TEMPLATE_INPUT_V3 = '{QUESTION}\nCorrect answer: {CORRECT_ANSWER}\nStudent wrong answer: {STUDENT_WRONG_ANSWER}'
def format_input_v3(row, wrong_choice):

    assert wrong_choice in "ABCD"
    # Extract values from the row
    question_text = row.get("QuestionText", "No question text provided")
    subject_name = row.get("SubjectName", "Unknown subject")
    construct_name = row.get("ConstructName", "Unknown construct")
    # Extract the correct and wrong answer text based on the choice
    correct_answer = row.get("CorrectAnswer", "Unknown")
    assert wrong_choice != correct_answer
    correct_answer_text = row.get(f"Answer{correct_answer}Text", "No correct answer text available")
    wrong_answer_text = row.get(f"Answer{wrong_choice}Text", "No wrong answer text available")

    # Construct the question format
    formatted_question = f"""Question: {question_text}
    
SubjectName: {subject_name}
ConstructName: {construct_name}"""

    # Return the extracted data
    ret = {
        "QUESTION": formatted_question,
        "CORRECT_ANSWER": correct_answer_text,
        "STUDENT_WRONG_ANSWER": wrong_answer_text,
        "MISCONCEPTION_ID": row.get('Misconception{wrong_choice}Id'),

        'ConstructName': row['ConstructName'],
        'SubjectName': row['SubjectName'],
        'QuestionText': row['QuestionText'],
        'CorrectAnswerText': correct_answer_text,
        'IncorrectAnswerText': wrong_answer_text,
    }
    ret["PROMPT"] = TEMPLATE_INPUT_V3.format(**ret)

    return ret


items = []
target_ids = []
for _, row in df_ret.iterrows():
    for choice in ['A', 'B', 'C', 'D']:
        if choice == row["CorrectAnswer"]:
            continue
        # if not IS_SUBMISSION and row[f'Misconception{choice}Id'] == -1:
        #     continue
            
        correct_col = f"Answer{row['CorrectAnswer']}Text"
        item = {'QuestionId_Answer': '{}_{}'.format(row['QuestionId'], choice)}
        item['Prompt'] = format_input_v3(row, choice)['PROMPT']

        item['ConstructName'] = row['ConstructName']
        item['SubjectName'] = row['SubjectName']
        item['QuestionText'] = row['QuestionText']
        correct_answer = row["CorrectAnswer"]
        item['CorrectAnswerText'] = row[f'Answer{correct_answer}Text']
        item['IncorrectAnswerText'] = row[f'Answer{choice}Text']
        
        items.append(item)
        target_ids.append(int(row.get(f'Misconception{choice}Id', -1)))
        
df_input = pd.DataFrame(items)

In [8]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'<instruct>{task_description}\n<query>{query}'

def get_detailed_example(task_description: str, query: str, response: str) -> str:
    return f'<instruct>{task_description}\n<query>{query}\n<response>{response}'

def get_new_queries(queries, query_max_len, examples_prefix, tokenizer):
    inputs = tokenizer(
        queries,
        max_length=query_max_len - len(tokenizer('<s>', add_special_tokens=False)['input_ids']) - len(
            tokenizer('\n<response></s>', add_special_tokens=False)['input_ids']),
        return_token_type_ids=False,
        truncation=True,
        return_tensors=None,
        add_special_tokens=False
    )
    prefix_ids = tokenizer(examples_prefix, add_special_tokens=False)['input_ids']
    suffix_ids = tokenizer('\n<response>', add_special_tokens=False)['input_ids']
    new_max_length = (len(prefix_ids) + len(suffix_ids) + query_max_len + 8) // 8 * 8 + 8
    new_queries = tokenizer.batch_decode(inputs['input_ids'])
    for i in range(len(new_queries)):
        new_queries[i] = examples_prefix + new_queries[i] + '\n<response>'
    return new_max_length, new_queries
task =  "Given a math multiple-choice problem with a student's wrong answer, retrieve the math misconceptions"
queries = [
    get_detailed_instruct(task, q) for q in df_input['Prompt']
]
documents = df_misconception_mapping['MisconceptionName'].tolist()
query_max_len, doc_max_len = 1024, 256
LORA_PATH = lora_path
tokenizer = AutoTokenizer.from_pretrained(qw14b_path)
examples_prefix = ''
new_query_max_len, new_queries = get_new_queries(queries, query_max_len, examples_prefix, tokenizer)


import json
with open('data.json', 'w') as f:
    data = {'texts': new_queries+ documents}
    f.write(json.dumps(data))

In [9]:
%%writefile run_embed.py
import argparse
import os
import json
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import peft

MAX_LENGTH = 2048

def last_token_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    left_padding = attention_mask[:, -1].sum() == attention_mask.shape[0]
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[
            torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths
        ]


def get_embeddings_in_batches(model, tokenizer, texts, max_length, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch_texts = texts[i : i + batch_size]
        batch_dict = tokenizer(
            batch_texts,
            max_length=max_length,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to("cuda")
        with torch.no_grad(), torch.amp.autocast("cuda"):
            outputs = model(**batch_dict)
            batch_embeddings = last_token_pool(
                outputs.last_hidden_state, batch_dict["attention_mask"]
            )
            batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1).cpu()
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)


def load_model_and_tokenizer(base_model_path, lora_path, load_in_4bit=True):
    model = AutoModel.from_pretrained(
        base_model_path,
        device_map=0,
        torch_dtype=torch.float16,
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        lora_path if lora_path else base_model_path
    )
    model.resize_token_embeddings(len(tokenizer))
    if lora_path:
        model = peft.PeftModel.from_pretrained(model, lora_path)
    return model, tokenizer


def main(args):
    output_file = args.input_text.replace(
        ".json", ".pt.fold.{}.{}.embed".format(*args.fold)
    )
    if os.path.exists(output_file):
        print(f"Output file {output_file} already exists. Skipping...")
        return
    model, tokenizer = load_model_and_tokenizer(
        args.base_model, args.lora_path, load_in_4bit=args.load_in_4bit
    )
    texts = json.load(open(args.input_text))["texts"][args.fold[0] :: args.fold[1]]
    embeddings = get_embeddings_in_batches(
        model,
        tokenizer,
        texts,
        max_length=MAX_LENGTH,
        batch_size=4,
    )
    text2embeds = {text: emb for text, emb in zip(texts, embeddings)}
    torch.save(text2embeds, output_file)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--base_model",
        type=str,
        default="Qwen/Qwen2.5-7B",
        help="Path to the base model",
    )
    parser.add_argument(
        "--lora_path",
        type=str,
        default=None,
        help="Path to the LoRA model",
    )
    parser.add_argument(
        "--input_text",
        type=str,
        default=".cache/data.json",
    )
    parser.add_argument(
        "--load_in_4bit",
        action="store_true",
        help="Load model in 4-bit mode",
    )
    parser.add_argument("--fold", nargs=2, type=int, default=[0, 1])
    args = parser.parse_args()
    if not os.path.exists(args.lora_path):
        args.lora_path = None
    main(args)

Overwriting run_embed.py


In [26]:
!sleep 1 & sleep 3

'sleep' is not recognized as an internal or external command,
operable program or batch file.
'sleep' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
%%writefile run.sh
lora_path = '/kaggle/input/2211-lora-14b/transformers/default/1'
cmd = f"(CUDA_VISIBLE_DEVICES=0 python run_embed.py --base_model {qw14b_path} --lora_path {lora_path} --input_text data.json --fold 0 2) & (CUDA_VISIBLE_DEVICES=1 python run_embed.py --base_model {qw14b_path} --lora_path {lora_path} --input_text data.json --fold 1 2)"
import os
os.system(cmd)

Writing run.sh


In [13]:
from glob import glob
import time
text_to_embed = {}
files = glob('*.pt*')
while len(files) != 2:
    time.sleep(1)
    files = glob('*.pt*')

time.sleep(3)    
for path in files:
    print(path)
    text_to_embed.update(torch.load(path))

KeyboardInterrupt: 

In [ ]:
len(text_to_embed)

In [ ]:
query_embeddings = torch.stack([text_to_embed[t] for t in new_queries])
doc_embeddings = torch.stack([text_to_embed[t] for t in documents])
query_embeddings.shape, doc_embeddings.shape

In [ ]:
eedi_11scores = query_embeddings @ doc_embeddings.T  # Shape: (M, N)
sorted_indices = torch.argsort(eedi_11scores,1, descending=True)[:,:35].tolist()

In [ ]:
df_input["MisconceptionId"] = [" ".join([str(x) for x in row]) for row in sorted_indices]
df_input[["QuestionId_Answer", "MisconceptionId"]].to_csv("submission.csv", index=False)

# **Random Forest Training&#8595;**

In [53]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)


# y_pred = rf_classifier.predict(X)
# print(classification_report(y_test, y_pred))

RandomForestClassifier(random_state=42)

# **Testing&#8595;**

In [54]:
def map_at_25(y_true, y_pred_probs, top_k=25):
    
    map_25 = 0.0
    for true_label, pred_prob in zip(y_true, y_pred_probs):
        # Get top_k predictions
        top_preds = np.argsort(pred_prob)[::-1][:top_k]
        
        if not true_label:
            continue
        
        score = 0.0
        hits = 0
        for i, pred in enumerate(top_preds, start=1):
            if pred == true_label:
                hits += 1
                score += hits / i  # Precision at i
        
        # Average Precision at 25
        map_25 += score / min(1, top_k)
    
    return map_25 / len(y_true)


In [55]:
y_val_pred_probs = rf_classifier.predict_proba(X_val)
y_val_true = list(y_val)

map25_score = map_at_25(y_val_true, y_val_pred_probs)
print(f"MAP@25 Score: {map25_score}")


MAP@25 Score: 0.0022291077409971054


In [28]:
y_pred_probs = rf_classifier.predict_proba(X_val)

# print predictions
for idx, (true_label, pred_prob) in enumerate(zip(y_val, y_pred_probs)):
    # Get top 25 predictions and probabilities
    top_preds = np.argsort(pred_prob)[::-1][:25]
    top_probs = pred_prob[top_preds]
    
    # Check if true is within top 25
    in_top_25 = true_label in top_preds
    
    print(f"Example {idx + 1}")
    print(f"True Label: {true_label}")
    print("Top 25 Predictions (Misconception ID: Probability):")
    for pred, prob in zip(top_preds, top_probs):
        print(f"ID {pred}: {prob:.4f}")
    print(f"True Label in Top 25: {in_top_25}\n")

    # Number of questions to print
    if idx == 10:  
        break

Example 1
True Label: 2030.0
Top 25 Predictions (Misconception ID: Probability):
True Label in Top 25: False

Example 2
True Label: 1794.0
Top 25 Predictions (Misconception ID: Probability):
True Label in Top 25: False

Example 3
True Label: 649.0
Top 25 Predictions (Misconception ID: Probability):
True Label in Top 25: False

Example 4
True Label: 2340.0
Top 25 Predictions (Misconception ID: Probability):
True Label in Top 25: False

Example 5
True Label: 1214.0
Top 25 Predictions (Misconception ID: Probability):
True Label in Top 25: False

Example 6
True Label: 617.0
Top 25 Predictions (Misconception ID: Probability):
True Label in Top 25: False

Example 7
True Label: 60.0
Top 25 Predictions (Misconception ID: Probability):
True Label in Top 25: False

Example 8
True Label: 1214.0
Top 25 Predictions (Misconception ID: Probability):
True Label in Top 25: False

Example 9
True Label: 1510.0
Top 25 Predictions (Misconception ID: Probability):
True Label in Top 25: False

Example 10
Tru